## Filter Operations

- &, |, ==
- ~

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Tutorial_03').getOrCreate()


In [5]:
spark

In [3]:
#Function to get all files in my directory

from os import listdir
from os.path import isfile, join

def get_file_from_directory(folder_dir='', file_name=''):
    filtered_list = []
    file_list = []

    try:
        if folder_dir != '':
            file_list = [f for f in listdir(folder_dir) if isfile(join(folder_dir, f))]

        if file_name != '':
            for arquivo in file_list:
                if file_name in arquivo:
                    filtered_list.append(file_name)
        else:
            return 'Insera o nome do arquivo a ser localizado!'
        
    except FileNotFoundError as error:
        print(f"Insira um caminho valido para a pasta de arquivos! \n Error: {error.filename}")
        
    except IndexError as index_error:
        print(f"Arquivo não localozado {index_error}, certifique-se de inserir o nome do arquivo.")
        
    try:
        return (f"{folder_dir}/{filtered_list[0]}", 'arquivo não localizado')[len(filtered_list) == 0]
    
    except IndexError:
        return 'Arquivo não localizado, talvez o nome do arquivo esteja errado.'

In [4]:
# set var values
folder='/home/toni/dev_spark/notebooks/files'
csv_file = 'movies.csv'

In [10]:
df_pyspark = spark.read.csv(get_file_from_directory(folder,csv_file), sep=';',header=True, inferSchema=True)
df_pyspark.show()

+------+--------------------+--------------------+----+------+-------------+
| movie|               title|              genres|year|Rating|Rotton Tomato|
+------+--------------------+--------------------+----+------+-------------+
|108583| Fawlty Towers (1975|              Comedy|null|     1|           54|
| 32898|Trip to the Moon,...|Action|Adventure|...|1902|     7|           80|
|  7065|Birth of a Nation...|           Drama|War|1915|     6|           92|
|  7243|Intolerance: Love...|               Drama|1915|     4|           82|
| 62383|20,000 Leagues Un...|                null|1915|     9|         null|
|  8511|      Immigrant, The|              Comedy|1917|     4|           59|
|  null|       Dog's Life, A|                null|1917|  null|           83|
| 72626|  Billy Blazes, Esq.|      Comedy|Western|1918|     2|           63|
|  6987|Cabinet of Dr. Ca...|Crime|Fantasy|Horror|1919|     4|           63|
| 25737|Golem, The (Golem...|                null|null|  null|         null|

## Filter

In [16]:
## Filter Rating > 6

df_pyspark.filter("Rating > 6").show()

## df_pyspark.filter(df_pyspark['Rating']> 6).show()

+-----+--------------------+--------------------+----+------+-------------+
|movie|               title|              genres|year|Rating|Rotton Tomato|
+-----+--------------------+--------------------+----+------+-------------+
|32898|Trip to the Moon,...|Action|Adventure|...|1902|     7|           80|
|62383|20,000 Leagues Un...|                null|1915|     9|         null|
|59945|  Mark of Zorro, The|Adventure|Romance...|1919|     8|           67|
| 3310|            Kid, The|        Comedy|Drama|1921|     8|           57|
|83096|  Haunted House, The|              Comedy|1920|     9|           61|
|83318|           Goat, The|              Comedy|1921|     8|           86|
+-----+--------------------+--------------------+----+------+-------------+



In [43]:
## filter null values
df_pyspark.filter((df_pyspark['Rating'] >= 7) & 
                  (df_pyspark['Rating'] <= 9) &
                 (df_pyspark["Rotton Tomato"].isNotNull())).show()
#%% [markdown]
## Many Approachs
# Approach - 1
    # df.filter("value is not null").show()

# Approach - 2
     # df.filter(col("value").isNotNull()).show()

# Approach - 3
     # df.filter(df["value"].isNotNull()).show()

# Approach - 4
    # df.filter(df.value.isNotNull()).show()

# Approach - 5
    # df.na.drop(subset=["value"]).show()

# Approach - 6
    # df.dropna(subset=["value"]).show()

#Source: https://medium.com/expedia-group-tech/start-your-journey-with-apache-spark-part-3-1ae77c05187


+-----+--------------------+--------------------+----+------+-------------+
|movie|               title|              genres|year|Rating|Rotton Tomato|
+-----+--------------------+--------------------+----+------+-------------+
|32898|Trip to the Moon,...|Action|Adventure|...|1902|     7|           80|
|59945|  Mark of Zorro, The|Adventure|Romance...|1919|     8|           67|
| 3310|            Kid, The|        Comedy|Drama|1921|     8|           57|
|83096|  Haunted House, The|              Comedy|1920|     9|           61|
|83318|           Goat, The|              Comedy|1921|     8|           86|
+-----+--------------------+--------------------+----+------+-------------+



In [47]:
df_pyspark.filter((df_pyspark['Rating'] == 7) |
                  (df_pyspark['Rating'] == 9) ).show()

+-----+--------------------+--------------------+----+------+-------------+
|movie|               title|              genres|year|Rating|Rotton Tomato|
+-----+--------------------+--------------------+----+------+-------------+
|32898|Trip to the Moon,...|Action|Adventure|...|1902|     7|           80|
|62383|20,000 Leagues Un...|                null|1915|     9|         null|
|83096|  Haunted House, The|              Comedy|1920|     9|           61|
+-----+--------------------+--------------------+----+------+-------------+



In [13]:
df_pyspark.filter("Rating > 6").select(['title','year','Rating']).show()

+--------------------+----+------+
|               title|year|Rating|
+--------------------+----+------+
|Trip to the Moon,...|1902|     7|
|20,000 Leagues Un...|1915|     9|
|  Mark of Zorro, The|1919|     8|
|            Kid, The|1921|     8|
|  Haunted House, The|1920|     9|
|           Goat, The|1921|     8|
+--------------------+----+------+



In [50]:
## Not operation '~'

df_pyspark.filter(~(df_pyspark['Rating'] == 9) & 
                 ~(df_pyspark['year'] == 1921)).show()

+-----+--------------------+--------------------+----+------+-------------+
|movie|               title|              genres|year|Rating|Rotton Tomato|
+-----+--------------------+--------------------+----+------+-------------+
|32898|Trip to the Moon,...|Action|Adventure|...|1902|     7|           80|
| 7065|Birth of a Nation...|           Drama|War|1915|     6|           92|
| 7243|Intolerance: Love...|               Drama|1915|     4|           82|
| 8511|      Immigrant, The|              Comedy|1917|     4|           59|
|72626|  Billy Blazes, Esq.|      Comedy|Western|1918|     2|           63|
| 6987|Cabinet of Dr. Ca...|Crime|Fantasy|Horror|1919|     4|           63|
|59945|  Mark of Zorro, The|Adventure|Romance...|1919|     8|           67|
|83322|           Boat, The|              Comedy|1920|     4|           65|
|25744|Haxan: Witchcraft...|  Documentary|Horror|1922|     6|           83|
+-----+--------------------+--------------------+----+------+-------------+

